In [ ]:
# Configuration
import subprocess
import numpy as np
import nibabel as nib
from sklearn.metrics import pairwise_distances
import sys
import os
sys.path.append("./mapalign")
from mapalign import embed
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pli
from glob import glob

In [ ]:
# os.chdir("/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/FunctionalGradients/") 
os.chdir("/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/FG_noBrainSmash/")

In [ ]:
import os
import time
import cupy as cp
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
start = time.time()
mempool = cp.get_default_memory_pool()

In [ ]:
### Self corrected function from mapalign because version conflict I think
def compute_diffusion_map_yp(L, alpha=0.5, n_components=None, diffusion_time=0,
                          skip_checks=False, overwrite=False):
    """Compute the diffusion maps of a symmetric similarity matrix

        L : matrix N x N
           L is symmetric and L(x, y) >= 0

        alpha: float [0, 1]
            Setting alpha=1 and the diffusion operator approximates the
            Laplace-Beltrami operator. We then recover the Riemannian geometry
            of the data set regardless of the distribution of the points. To
            describe the long-term behavior of the point distribution of a
            system of stochastic differential equations, we can use alpha=0.5
            and the resulting Markov chain approximates the Fokker-Planck
            diffusion. With alpha=0, it reduces to the classical graph Laplacian
            normalization.

        n_components: int
            The number of diffusion map components to return. Due to the
            spectrum decay of the eigenvalues, only a few terms are necessary to
            achieve a given relative accuracy in the sum M^t.

        diffusion_time: float >= 0
            use the diffusion_time (t) step transition matrix M^t

            t not only serves as a time parameter, but also has the dual role of
            scale parameter. One of the main ideas of diffusion framework is
            that running the chain forward in time (taking larger and larger
            powers of M) reveals the geometric structure of X at larger and
            larger scales (the diffusion process).

            t = 0 empirically provides a reasonable balance from a clustering
            perspective. Specifically, the notion of a cluster in the data set
            is quantified as a region in which the probability of escaping this
            region is low (within a certain time t).

        skip_checks: bool
            Avoid expensive pre-checks on input data. The caller has to make
            sure that input data is valid or results will be undefined.

        overwrite: bool
            Optimize memory usage by re-using input matrix L as scratch space.

        References
        ----------

        [1] https://en.wikipedia.org/wiki/Diffusion_map
        [2] Coifman, R.R.; S. Lafon. (2006). "Diffusion maps". Applied and
        Computational Harmonic Analysis 21: 5-30. doi:10.1016/j.acha.2006.04.006
    """

    import numpy as np
    import scipy.sparse as sps

    use_sparse = False
    if sps.issparse(L):
        use_sparse = True

    if not skip_checks:
        # Original: from sklearn.manifold.spectral_embedding_ import _graph_is_connected
        from sklearn.manifold._spectral_embedding import _graph_is_connected
        # from sklearn.manifold.spectral_embedding import _graph_is_connected
        if not _graph_is_connected(L):
            raise ValueError('Graph is disconnected')

    ndim = L.shape[0]
    if overwrite:
        L_alpha = L
    else:
        L_alpha = L.copy()

    if alpha > 0:
        # Step 2
        d = np.array(L_alpha.sum(axis=1)).flatten()       # 行相加求和：axis=0, 表示列。axis=1, 表示行. flatten: 返回一个一维数组。
        d_alpha = np.power(d, -alpha)                     # 0.5次方：np.power 求n次方
        if use_sparse:
            L_alpha.data *= d_alpha[L_alpha.indices]
            L_alpha = sps.csr_matrix(L_alpha.transpose().toarray())
            L_alpha.data *= d_alpha[L_alpha.indices]
            L_alpha = sps.csr_matrix(L_alpha.transpose().toarray())
        else:
            L_alpha = d_alpha[:, np.newaxis] * L_alpha    # 插入新维度
            L_alpha = L_alpha * d_alpha[np.newaxis, :]

    # Step 3
    d_alpha = np.power(np.array(L_alpha.sum(axis=1)).flatten(), -1)
    if use_sparse:
        L_alpha.data *= d_alpha[L_alpha.indices]
    else:
        L_alpha = d_alpha[:, np.newaxis] * L_alpha

    M = L_alpha

    from scipy.sparse.linalg import eigsh, eigs

    # Step 4
    func = eigs
    if n_components is not None:
        lambdas, vectors = func(M, k=n_components + 1)
    else:
        # 求矩阵M的k个特征值和特征向量, 
        # lamadas：ndarray k个特征值的数组. 
        # vector：ndarray k个特征向量的数组. v[:, i]是对应于特征值w [i]的特征向量。
        lambdas, vectors = func(M, k=max(2, int(np.sqrt(ndim))))        
    del M

    if func == eigsh:
        lambdas = lambdas[::-1]
        vectors = vectors[:, ::-1]
    else:
        lambdas = np.real(lambdas)
        vectors = np.real(vectors)
        lambda_idx = np.argsort(lambdas)[::-1]
        lambdas = lambdas[lambda_idx]
        vectors = vectors[:, lambda_idx]

    return _step_5(lambdas, vectors, ndim, n_components, diffusion_time)

def _step_5(lambdas, vectors, ndim, n_components, diffusion_time):
    """
    This is a helper function for diffusion map computation.

    The lambdas have been sorted in decreasing order.
    The vectors are ordered according to lambdas.

    """
    psi = vectors/vectors[:, [0]]
    diffusion_times = diffusion_time
    if diffusion_time == 0:
        diffusion_times = np.exp(1. -  np.log(1 - lambdas[1:])/np.log(lambdas[1:]))
        lambdas = lambdas[1:] / (1 - lambdas[1:])
    else:
        lambdas = lambdas[1:] ** float(diffusion_time)
    lambda_ratio = lambdas/lambdas[0]
    threshold = max(0.05, lambda_ratio[-1])

    n_components_auto = np.amax(np.nonzero(lambda_ratio > threshold)[0])
    n_components_auto = min(n_components_auto, ndim)
    if n_components is None:
        n_components = n_components_auto
    embedding = psi[:, 1:(n_components + 1)] * lambdas[:n_components][None, :]

    result = dict(lambdas=lambdas, vectors=vectors,
                  n_components=n_components, diffusion_time=diffusion_times,
                  n_components_auto=n_components_auto)
    return embedding, result


# Step 1. Calculate the gradient

In [ ]:
## 1. Load FC matrix
Grey_file = nib.load("HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii")
axis2 = Grey_file.header.get_axis(1)
cerebellum_mask_ind = np.append(np.where(axis2.__dict__['_name'] == 'CIFTI_STRUCTURE_CEREBELLUM_LEFT'), 
                                np.where(axis2.__dict__['_name'] == 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT'))
# FC_all = nib.load('HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii').get_fdata() # 91282*91282
# FC_cere2extra_1 = FC_all[cerebellum_mask_ind, :]
# FC_cere2extra = np.delete(FC_cere2extra_1,cerebellum_mask_ind, axis=1)
FC_filename = 'HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr_cere2extra_17853'
# np.save(FC_filename, FC_cere2extra) #17853*73429, 

## Calculate the gradient
# dcon = np.tanh(np.load(str(FC_filename) + '.npy'))
# perc = np.array([np.percentile(x, 90) for x in dcon])                     
# for j in range(dcon.shape[0]):                                            
#     print ("Row %d" % j)
#     dcon[j, dcon[j,:] < perc[j]] = 0
# dcon[dcon < 0] = 0
# aff = 1 - pairwise_distances(dcon, metric = 'cosine')     
# aff_filename = 'cosine_affinity_cere2extra_17853.npy'
# np.save(aff_filename, aff)
# aff = np.load(aff_filename)
# emb, res = compute_diffusion_map_yp(aff, alpha = 0.5)      
# emb_filename = 'embedding_dense_cere2extra_emb_17853.npy'     
# res_filename = 'embedding_dense_cere2extra_res_17853.npy'           
# np.save(emb_filename, emb)                   
# np.save(res_filename, res)
# # This is for Normalizetion
# emb_i = -1 * emb
# emb_i_filename = 'embedding_dense_cere2extra_emb_i_17853.npy'   
# np.save(emb_i_filename, emb_i) 

# emb_z = pd.DataFrame()
# emb_s = pd.DataFrame()
# for j in range(len(emb.T)):
#     print(j)
#     _range = emb_i.T[j].max() - emb_i.T[j].min()
#     emb_z[j] = (emb_i.T[j] -  emb_i.T[j].min()) / _range
#     emb_s[j] = ((emb_i.T[j] -  emb_i.T[j].min()) / _range)+1
# emb_z_filename = 'embedding_dense_cere2extra_emb_z_17853.npy'   
# np.save(emb_z_filename, emb_z.values) 
# emb_s_filename = 'embedding_dense_cere2extra_emb_s_17853.npy'   
# np.save(emb_s_filename, emb_s.values) 

# Step 2. Show the gradient

In [ ]:
# Try using Suitpy
import SUITPy as suit
plot_data = suit.flatmap.vol_to_surf('result_cere2extra_gradient1__z_17853_nifti.nii',space='SUIT', stats='nanmean')
suit.flatmap.plot(data=plot_data, new_figure=True, colorbar=True, cmap='jet') # , cscale=[np.min(plot_data), np.max(plot_data)]

In [ ]:
## 2. Save out and show
from tqdm import tqdm
for emb_idx in ['', '_i', '_z', '_s']:
    print(emb_idx)
    emb_name = 'embedding_dense_cere2extra_emb' + str(emb_idx) + '_17853.npy'
    emb_cur = np.load(emb_name)
    cope1_file = nib.load("cope1.dtseries.nii")
    output = np.zeros([91282])
    for j in tqdm(range(len(emb_cur.T))):
        dscalar_name = 'result_cere2extra_gradient' + str(j+1) + '_' + str(emb_idx) + '_17853.dscalar.nii'
        nifti_name = 'result_cere2extra_gradient' + str(j+1) + '_' + str(emb_idx) + '_17853_nifti.nii'
        image_name = 'result_cere2extra_gradient' + str(j+1) + '_' + str(emb_idx) + '_17853.png'
        pdf_name = 'result_cere2extra_gradient' + str(j+1) + '_' + str(emb_idx) + '_17853.pdf'
        
        # dscalar file
        output[cerebellum_mask_ind] = emb_cur [:,j]
        img_test = nib.cifti2.Cifti2Image(output.reshape([1,-1]),cope1_file.header)
        img_test.to_filename(dscalar_name)
        
        # nifti file
        cmd_5 = "wb_command -cifti-separate " + str(dscalar_name) + " COLUMN -volume-all " + str(nifti_name)
        print(cmd_5)
        subprocess.check_output(cmd_5, shell=True);
        
        # show in png
        cmd_6 = "scp " + str(nifti_name) +' image_cerebellumonly_nifti.nii'
        subprocess.check_output(cmd_6, shell=True);
        subprocess.check_output('bash call_matlab_cerebellumonly.sh', shell=True);
        image = plt.imread('image_cerebellumonly.png')  
        plt.imshow(image)
        plt.show()
        
        # rename and svae in pdf
        cmd_7 = "scp image_cerebellumonly.png " + str(image_name)
        subprocess.check_output(cmd_7, shell=True);
        cmd_8 = "convert image_cerebellumonly.png " + str(pdf_name)
        subprocess.check_output(cmd_8, shell=True);
    

In [ ]:
## 2. Save out and show
# for emb_idx in ['', '_i', '_z', '_s']:
emb_idx = '_z'
print(emb_idx)
emb_name = 'embedding_dense_cere2extra_emb' + str(emb_idx) + '_17853.npy'
emb_cur = np.load(emb_name)
cope1_file = nib.load("cope1.dtseries.nii")
output = np.zeros([91282])
# for j in range(len(emb_cur.T)):
j = 0
dscalar_name = 'result_cere2extra_gradient' + str(j+1) + '_' + str(emb_idx) + '_17853.dscalar.nii'
nifti_name = 'result_cere2extra_gradient' + str(j+1) + '_' + str(emb_idx) + '_17853_nifti.nii'
image_name = 'result_cere2extra_gradient' + str(j+1) + '_' + str(emb_idx) + '_17853.png'
pdf_name = 'result_cere2extra_gradient' + str(j+1) + '_' + str(emb_idx) + '_17853.pdf'

# dscalar file
output[cerebellum_mask_ind] = emb_cur [:,j]
img_test = nib.cifti2.Cifti2Image(output.reshape([1,-1]),cope1_file.header)
img_test.to_filename(dscalar_name)

# nifti file
cmd_5 = "wb_command -cifti-separate " + str(dscalar_name) + " COLUMN -volume-all " + str(nifti_name)
print(cmd_5)
subprocess.check_output(cmd_5, shell=True);

# show in png
cmd_6 = "scp " + str(nifti_name) +' image_cerebellumonly_nifti.nii'
subprocess.check_output(cmd_6, shell=True);
subprocess.check_output('bash call_matlab_cerebellumonly.sh', shell=True);
image = plt.imread('image_cerebellumonly.png')  
plt.imshow(image)
plt.show()

# rename and svae in pdf
cmd_7 = "scp image_cerebellumonly.png " + str(image_name)
subprocess.check_output(cmd_7, shell=True);
cmd_8 = "convert image_cerebellumonly.png " + str(pdf_name)
subprocess.check_output(cmd_8, shell=True);


# Step 3. Confirm SA

## Quantiative demonstation

In [ ]:
## Configuration
import pandas
import pandas as pd
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import seaborn as sns
from glob import glob
from scipy import stats
from nilearn import datasets, image, input_data, plotting
import scipy
from scipy import io
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
git_dir = '/n02dat01/users/ypwang/Gradient/GeneticGradient/Scripts/'
import sys
sys.path.insert(0,git_dir)
%load_ext autoreload
%autoreload 1
%aimport FunctionLibrary


In [ ]:
def RGB_to_Hex(rgb):
    RGB = rgb.split(',')
    color = '#'
    for i in RGB:
        num = int(i)
        color += str(hex(num))[-2:].replace('x', '0').upper()
    return color
yeo_dir = '/n02dat01/users/ypwang/Gradient/Tool/Atlas/Yeo2011_fcMRI_clustering/1000subjects_reference/'  

# color_7 = pd.read_csv("/n01dat01/ypwang/AHBA/CerebellarGeneFCCorrelation/Reference_files/7NetworksColors.csv")
mat_struct = scipy.io.loadmat(f"{yeo_dir}/7NetworksColors.mat")
color_7 = pd.DataFrame(mat_struct['colors']).drop(0)
color_7.columns = ['R', 'G', 'B']
index_7 = pd.read_csv("/n01dat01/ypwang/AHBA/CerebellarGeneFCCorrelation/Reference_files/7network_names.csv")
# index_7 = pd.read_csv(f"{yeo_dir}/7NetworksOrderedNames.csv", index_col=0)
color_7_hex = [RGB_to_Hex(str(tuple(color_7.iloc[i,:])).replace("(","").replace(")","")) for i in range(len(color_7))]
color_7_hex = pd.DataFrame(color_7_hex)
color_7.index = color_7_hex.index 
color_7 = pd.merge(color_7_hex,color_7,left_index=True,right_index=True)
color_7 = pd.merge(index_7,color_7,left_index=True,right_index=True)
color_7.columns = ['Name', 'Hex', 'R', 'G', 'B']

order = ['SomMot', 'Vis', 'DorsAttn', 'VentAttn', 'Cont', 'Limbic', 'Default']
    
from nilearn.input_data import NiftiLabelsMasker
Net_7 = nib.load("/n02dat01/users/ypwang/SCZ/SCZ_202310/Data/Info/Atlas/yeo_atlas/cere_2mm_17.nii.gz")
Net_7 = NiftiLabelsMasker(labels_img=Net_7, standardize=False, strategy="mean")
Net_7.fit()
Net_7_data = []

mat_struct = scipy.io.loadmat(f"{yeo_dir}/7NetworksColors.mat")
color_7 = pd.DataFrame(mat_struct['colors']).drop(0)
color_7.columns = ['R', 'G', 'B']
index_7 = pd.read_csv("/n01dat01/ypwang/AHBA/CerebellarGeneFCCorrelation/Reference_files/7network_names.csv")
# index_7 = pd.read_csv(f"{yeo_dir}/7NetworksOrderedNames.csv", index_col=0)
color_7_hex = [RGB_to_Hex(str(tuple(color_7.iloc[i,:])).replace("(","").replace(")","")) for i in range(len(color_7))]
color_7_hex = pd.DataFrame(color_7_hex)
color_7.index = color_7_hex.index 
color_7 = pd.merge(color_7_hex,color_7,left_index=True,right_index=True)
color_7 = pd.merge(index_7,color_7,left_index=True,right_index=True)
color_7.columns = ['Name', 'Hex', 'R', 'G', 'B']

In [ ]:
import pandas as pd
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# Load data
net = 7
atlas_file = nib.load(f'/n14dat01/lchai/HCP_MIN/cere_2mm_{net}.nii.gz')
label_values = np.unique(atlas_file.get_fdata())
nii_file = nib.load('result_cerebellumonly_gradient1__17853_nifti.nii')
FG_net7 = nii_file.get_fdata()

# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

for x in label_values:
    mask = (atlas_file.get_fdata() == x)
    voxel_data = FG_net7[mask]
    label_data = np.repeat(x, np.sum(mask))
    FG_cur = pd.DataFrame({'Voxel Data': voxel_data, 'Label': label_data})
    
    # Concatenate the current DataFrame with the result DataFrame
    result_df = pd.concat([result_df, FG_cur])

label_name_mapping = {
    1: 'Vis',
    2: 'SomMot',
    3: 'DorsAttn',
    4: 'VentAttn',
    5: 'Limbic',
    6: 'Cont',
    7: 'Default'
}

# Add a new column 'label_name' based on the mapping
result_df['label_name'] = result_df['Label'].map(label_name_mapping)
# Boxplot code
order_cur = ['SomMot', 'Vis', 'DorsAttn', 'VentAttn', 'Cont', 'Limbic', 'Default']
# color_7 = [color_7.loc[color_7.loc[:,'Name']==i,'Hex'].squeeze() for i in order_cur]
color_reorder_cur = [color_7.loc[color_7.loc[:, 'Name'] == i, 'Hex'].squeeze() for i in order_cur]

fig, ax = plt.subplots(figsize=(10, 8))

bx = ax.boxplot([result_df[result_df.loc[:, 'label_name'] == i].loc[:, 'Voxel Data'].values for i in order_cur],
                labels=order_cur, notch=0, vert=True, patch_artist=True, showfliers=False, showmeans=True,
                meanprops={"marker": "D", "markerfacecolor": "white", "linewidth": 2, "markeredgecolor": "grey"},
                medianprops={"color": "white", "linewidth": 1},
                whiskerprops={"linewidth": 2.5},
                capprops={"linewidth": 3})

for index, bp in enumerate(bx["boxes"]):
    bp.set_facecolor(color_reorder_cur[index])
    bp.set_edgecolor(color_reorder_cur[index])
    bp.set_alpha(0.8)

for index, bp in enumerate(bx["whiskers"]):
    bp.set_color(color_reorder_cur[int(index / 2)])
    bp.set_alpha(0.8)

for index, bp in enumerate(bx["caps"]):
    bp.set_color(color_reorder_cur[int(index / 2)])
    bp.set_alpha(0.8)

ax.axhline(0, linestyle='--', color='grey')  # , color='k'
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_ylabel("FG", fontsize=20)
ax.tick_params(axis='both', labelsize=14)
# ax.set_xlabel('Networks', fontsize=16)
ax.spines['bottom'].set_linewidth(1.6)
ax.spines['left'].set_linewidth(1.6)

plt.savefig('./R2.2/FG_Network_7.png', dpi=700, bbox_inches='tight')
plt.show()


In [ ]:
def RGB_to_Hex(rgb):
    RGB = rgb.replace("'", '').split(',')
    color = '#'
    for i in RGB:
        num = int(i)
        color += str(hex(num))[-2:].replace('x', '0').upper()
    return color

data = []
with open("/n02dat01/users/ypwang/AHBA/Github_20220301/Reference_files/MDTB_10Regions_Color.txt", 'r', encoding='utf8') as f:
    for i in f:
        data.append([j for j in i.split()])

color_10 = pd.DataFrame(data).iloc[:, 1:]
color_10.columns = ['R', 'G', 'B']
index_10 = pd.read_csv("/n02dat01/users/ypwang/AHBA/Github_20220301/Reference_files/MDTB_10network_names.CSV")
color_10_hex = [RGB_to_Hex(','.join(color_10.iloc[i, :])) for i in range(len(color_10))]
color_10_hex = pd.DataFrame(color_10_hex)
color_10.index = color_10_hex.index
color_10 = pd.merge(color_10_hex, color_10, left_index=True, right_index=True)
color_10 = pd.merge(index_10, color_10, left_index=True, right_index=True)
color_10.columns = ['Name', 'Hex', 'R', 'G', 'B']


In [ ]:
index_10

In [ ]:
import pandas as pd
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# Load data
atlas_file = nib.load("/n02dat01/users/ypwang/SCZ/SCZ_202310/Data/Info/Atlas/MDTB_atlas/MDTB_10Regions_MNI_2MM.nii.gz")
label_values = np.unique(atlas_file.get_fdata())
nii_file = nib.load('result_cerebellumonly_gradient1__17853_nifti.nii')
FG_net10 = nii_file.get_fdata()

# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

for x in label_values:
    mask = (atlas_file.get_fdata() == x)
    voxel_data = FG_net10[mask]
    label_data = np.repeat(x, np.sum(mask))
    FG_cur = pd.DataFrame({'VoxelData': voxel_data, 'Label': label_data})
    
    # Concatenate the current DataFrame with the result DataFrame
    result_df = pd.concat([result_df, FG_cur])

label_name_mapping = {
    1: 'Left_Hand_Press',
    2: 'Right_Hand_Press',
    3: 'Saccades',
    4: 'Action_observation',
    5: 'Divided_Attention_Mental',
    6: 'Divided_Attention_Verbal',
    7: 'Narrative',
    8: 'Word_Comprehension',  # 请根据实际情况修改
    9: 'Verbal_Fluency',
    10: 'Autobiographical_Recall'
}

# Add a new column 'label_name' based on the mapping
result_df['label_name'] = result_df['Label'].map(label_name_mapping)
result_df['label_name'] = result_df['label_name'].astype(str)
order_df = pd.DataFrame(np.zeros([len(np.unique(result_df.loc[:, 'label_name'])), len(result_df.T)]))
order_df.index = np.unique(result_df.label_name)
order_df['median'] = [np.median(result_df.loc[result_df.loc[:,'label_name'] == x,'VoxelData']) for x in np.unique(result_df.label_name)]
# print(order_df.sort_values(by='mean', ascending=False))
order_cur = order_df.sort_values(by='median', ascending=True).index.values
order_cur = order_cur[order_cur != 'nan']
color_reorder_cur = [color_10.loc[color_10.loc[:,'Name']==i,'Hex'].squeeze() for i in order_cur]
# Boxplot code
# order_cur = ['SomMot', 'Vis', 'DorsAttn', 'VentAttn', 'Cont', 'Limbic', 'Default']
# color_7 = [color_7.loc[color_7.loc[:,'Name']==i,'Hex'].squeeze() for i in order_cur]
# color_10 = [color_7.loc[color_7.loc[:, 'Name'] == i, 'Hex'].squeeze() for i in order_cur]


In [ ]:

fig, ax = plt.subplots(figsize=(10, 8))

bx = ax.boxplot([result_df[result_df.loc[:, 'label_name'] == i].loc[:, 'VoxelData'].values for i in order_cur],
                labels=[label.replace('_', '\n') for label in order_cur], notch=0, vert=True, patch_artist=True, showfliers=False, showmeans=True,
                meanprops={"marker": "D", "markerfacecolor": "white", "linewidth": 2, "markeredgecolor": "grey"},
                medianprops={"color": "white", "linewidth": 1},
                whiskerprops={"linewidth": 2.5},
                capprops={"linewidth": 3})

for index, bp in enumerate(bx["boxes"]):
    bp.set_facecolor(color_reorder_cur[index])
    bp.set_edgecolor(color_reorder_cur[index])
    bp.set_alpha(0.8)

for index, bp in enumerate(bx["whiskers"]):
    bp.set_color(color_reorder_cur[int(index / 2)])
    bp.set_alpha(0.8)

for index, bp in enumerate(bx["caps"]):
    bp.set_color(color_reorder_cur[int(index / 2)])
    bp.set_alpha(0.8)

ax.axhline(0, linestyle='--', color='grey')  # , color='k'
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_ylabel("FG", fontsize=20)
ax.tick_params(axis='both', labelsize=14)
# ax.set_xlabel('Networks', fontsize=16)
ax.spines['bottom'].set_linewidth(1.6)
ax.spines['left'].set_linewidth(1.6)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='center', va='top', multialignment='right')
plt.savefig('./R2.2/FG_Network_10.png', dpi=700, bbox_inches='tight')
plt.show()


## 100 cerebellar FG sorted ROI

In [ ]:
def disbtw(surf, aa):
    surf= np.array(surf)
    aa = np.expand_dims(aa,axis=0)
    dis = np.sqrt(((surf-aa)**2).sum(axis=1))
    return dis

def Grid2world(GridList, affine):
    """
    trans the grid coords into the corresponding MNI coords in MNI space
    GridList shape: N*3, affine shape 4*4
    """
    GridList = np.array(GridList)
    length = len(GridList)
    affine = np.array(affine)
    Grid_coord = np.concatenate([GridList, np.ones([length, 1])], axis=1)
    MNI_coord = Grid_coord.dot(affine.T)      
    MNI_coord = MNI_coord.round(2)
    return MNI_coord[:, :3]

In [ ]:
import numpy as np
import nibabel as nib

# Load the gradient file
gradient_file = nib.load('result_cerebellumonly_gradient1__17853_nifti.nii')
gradient = gradient_file.get_data()
affine = gradient_file.affine

# Find non-zero values and their indices in the gradient
grid_list = np.nonzero(gradient)
grad_values = gradient[grid_list]

# Split gradient values into 10 bins
bins = np.linspace(np.min(grad_values), np.max(grad_values), 100+1)
digitized = np.digitize(grad_values, bins)

# Create 10 ROIs based on the bins

for i in range(1, 100+1):
    # Get indices for the current bin
    bin_indices = grid_list[0][digitized == i], grid_list[1][digitized == i], grid_list[2][digitized == i]

    # Create an empty array for the ROI
    roi = np.zeros_like(gradient)

    # Set the values in the ROI array to a specific range (e.g., 1 to 10)
    roi[bin_indices] = i
    # Fill the ROI array with values from the current bin
    # roi[bin_indices] = gradient[bin_indices]

    # Create a new NIfTI file for the ROI
    roi_nifti = nib.Nifti1Image(roi, affine)
    nib.save(roi_nifti, f'./R2.2/roi_100_{i}.nii')

# load nii to get coords

for i in range(1, 100+1):
    data = nib.load(f'./R2.2/roi_100_{i}.nii').get_fdata()
    coord = np.array(np.where(data==i)).T
    pd.DataFrame(coord).to_csv(f'./R2.2/roi_100_{i}_res2mm_grid.txt', sep='\t', index=False, header=False)    

Generate the list for matlab type input

In [ ]:
# Generate the list for matlab type input
Sample_List=[]

for i in range(1, 100+1):
    Sample_List.append(f'roi_100_{i}')
    print(len(Sample_List))
    pd.DataFrame(Sample_List).to_csv(f'./R2.2/ROIList_100_res2mm.txt', sep='\t', index=False, header=False)


Show the cerebellum ROI

In [ ]:
import numpy as np
import nibabel as nib

# Load the gradient file
gradient_file = nib.load('result_cerebellumonly_gradient1__17853_nifti.nii')
gradient = gradient_file.get_data()
affine = gradient_file.affine

# Find non-zero values and their indices in the gradient
grid_list = np.nonzero(gradient)
grad_values = gradient[grid_list]

# Split gradient values into 10 bins
bins = np.linspace(np.min(grad_values), np.max(grad_values), 100+1)
digitized = np.digitize(grad_values, bins)

# Create 10 ROIs based on the bins
roi = np.zeros_like(gradient)
for i in range(1, 100+1, 10):
    # Get indices for the current bin
    bin_indices = grid_list[0][digitized == i], grid_list[1][digitized == i], grid_list[2][digitized == i]
    roi[bin_indices] = i
roi_nifti = nib.Nifti1Image(roi, affine)
nib.save(roi_nifti, f'./R2.2/roi_100_10together.nii')


可视化到大脑的连接值

In [ ]:
import nibabel as nib
from nilearn import image, surface, plotting, datasets
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.pyplot as plt
from nilearn import datasets
fsaverage = datasets.fetch_surf_fsaverage()

mesh_L = "/n02dat01/users/ypwang/Gradient/Tool/Atlas/BN_Atlas_freesurfer/BN_Atlas_freesurfer/fsaverage/fsaverage_LR32k/fsaverage.L.inflated.32k_fs_LR.surf.gii"
mesh_R = "/n02dat01/users/ypwang/Gradient/Tool/Atlas/BN_Atlas_freesurfer/BN_Atlas_freesurfer/fsaverage/fsaverage_LR32k/fsaverage.R.inflated.32k_fs_LR.surf.gii"

fig, axes = plt.subplots(10, 4, figsize=(12, 40), subplot_kw={"projection": "3d"})

for i,index in enumerate([10,30,60,90]):
    print(f'ROI{index}')
    volume_img = nib.load(f"./R2.2/lesion_network/ROI100/roi_100_{index}_t_fwe_0.001.nii.gz")
    
    surface_L = surface.vol_to_surf(volume_img, fsaverage['pial_left'])
    surface_L[surface_L==0]=np.nan
    plotting.plot_surf_stat_map(fsaverage['pial_left'], surface_L, 
                                cmap='RdBu_r',
                                vmax=20,
                                colorbar=False,
                                hemi='left',
                                bg_map=fsaverage.sulc_left, 
                                axes=axes[i, 0])
    plotting.plot_surf_stat_map(fsaverage['pial_left'], surface_L, 
                                cmap='RdBu_r',
                                vmax=20,
                                colorbar=False,
                                hemi='left',
                                bg_map=fsaverage.sulc_left, 
                                view='medial',
                                axes=axes[i, 1])
    
    surface_R = surface.vol_to_surf(volume_img, fsaverage['pial_right'])
    surface_R[surface_R==0]=np.nan
    plotting.plot_surf_stat_map(fsaverage['pial_right'], surface_R, 
                                cmap='RdBu_r',
                                vmax=20,
                                colorbar=False,
                                hemi='right',
                                bg_map=fsaverage.sulc_right, 
                                axes=axes[i, 2])
    plotting.plot_surf_stat_map(fsaverage['pial_right'], surface_R, 
                                cmap='RdBu_r',
                                vmax=20,
                                colorbar=False,
                                hemi='right',
                                bg_map=fsaverage.sulc_right, 
                                view='medial',
                                axes=axes[i, 3])

# Remove unused subplots
for i in range(10, 100+1, 10):
    axes[i//10-1, 1].axis('off')

# Adjust spacing between subplots
plt.tight_layout()
plt.savefig(f'./R2.2/Output/roi_100_FC2cort_fwe.png',dpi=500, bbox_inches='tight')
# Show the plot
plt.show()

In [ ]:
## Configuration
import pandas
import pandas as pd
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import seaborn as sns
from glob import glob
from scipy import stats
from nilearn import datasets, image, input_data, plotting
import scipy
from scipy import io
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
git_dir = '/n02dat01/users/ypwang/Gradient/GeneticGradient/Scripts/'
import sys
sys.path.insert(0,git_dir)
%load_ext autoreload
%autoreload 1
%aimport FunctionLibrary

In [ ]:
def RGB_to_Hex(rgb):
    RGB = rgb.split(',')
    color = '#'
    for i in RGB:
        num = int(i)
        color += str(hex(num))[-2:].replace('x', '0').upper()
    return color
yeo_dir = '/n02dat01/users/ypwang/Gradient/Tool/Atlas/Yeo2011_fcMRI_clustering/1000subjects_reference/'  

# color_7 = pd.read_csv("/n01dat01/ypwang/AHBA/CerebellarGeneFCCorrelation/Reference_files/7NetworksColors.csv")
mat_struct = scipy.io.loadmat(f"{yeo_dir}/7NetworksColors.mat")
color_7 = pd.DataFrame(mat_struct['colors']).drop(0)
color_7.columns = ['R', 'G', 'B']
index_7 = pd.read_csv("/n01dat01/ypwang/AHBA/CerebellarGeneFCCorrelation/Reference_files/7network_names.csv")
# index_7 = pd.read_csv(f"{yeo_dir}/7NetworksOrderedNames.csv", index_col=0)
color_7_hex = [RGB_to_Hex(str(tuple(color_7.iloc[i,:])).replace("(","").replace(")","")) for i in range(len(color_7))]
color_7_hex = pd.DataFrame(color_7_hex)
color_7.index = color_7_hex.index 
color_7 = pd.merge(color_7_hex,color_7,left_index=True,right_index=True)
color_7 = pd.merge(index_7,color_7,left_index=True,right_index=True)
color_7.columns = ['Name', 'Hex', 'R', 'G', 'B']

order = ['SomMot', 'Vis', 'DorsAttn', 'VentAttn', 'Cont', 'Limbic', 'Default']
    
from nilearn.input_data import NiftiLabelsMasker
Net_7 = nib.load("/n02dat01/users/ypwang/Gradient/Tool/Atlas/Yeo2011_fcMRI_clustering/1000subjects_reference/Yeo_JNeurophysiol11_SplitLabels/MNI152/Yeo2011_7Networks_N1000.split_components.FSL_MNI152_2mm.nii.gz")
Net_7 = NiftiLabelsMasker(labels_img=Net_7, standardize=False, strategy="mean")
Net_7.fit()
Net_7_data = []

In [ ]:
decile = 10
N_name = pd.read_csv(f"/n01dat01/ypwang/AHBA/CerebellarGeneFCCorrelation/Data/Yeo_JNeurophysiol11_SplitLabels/Yeo_JNeurophysiol11_SplitLabels/Yeo2011_7networks_N1000.split_components.glossary.csv")
    
# plt.close()
# fig,ax= plt.subplots(10,1, figsize=(10, 60)) #, gridspec_kw={'height_ratios': [1, 1.5, 1, 1.5, 1, 1.5, 1, 1.5]})
# plt.tight_layout(pad=0, w_pad=0, h_pad=0.9)
# fig.subplots_adjust(hspace=0.4)
for id,index in enumerate(range(1, decile+1)):
    FC_name =  nib.load(f"./R2.2/lesion_network/roi_{decile}_{index}_t_fwe_0.001.nii.gz")
    FC_net7 = np.array(Net_7.transform(FC_name)).squeeze()

    N_comparision = N_name.iloc[1:,:]
    R = pd.DataFrame(FC_net7)
    R.index = N_comparision.index
    R.columns = ['R']
    N_comparision = pd.concat([R, R, R, N_comparision], axis= 1)

    # net 7
    for i in range(len(N_comparision)):
        N_comparision.iloc[i,1] = N_comparision.iloc[i,3].split('_')[-2]
        if (N_comparision.iloc[i,1] in ['LH', 'RH']):
            N_comparision.iloc[i,1] = N_comparision.iloc[i,3].split('_')[-1]
        if ('SalVentAttn' in N_comparision.iloc[i,1]):
            N_comparision.iloc[i,1] = N_comparision.iloc[i,1].replace('SalVentAttn', 'VentAttn')
    N_comparision.columns = ['Net_index', 'Name', 'R', 'Label_Name', 'Network_Name','Full_component_name']

    for i in np.unique(N_comparision.loc[:,'Name'].values):
        for j in range(len(N_comparision)):
            if N_comparision.iloc[j,1]== i:
                N_comparision.iloc[j,0]= int(index_7[index_7.loc[:,'None']==i].index.values)
    N_comparision = N_comparision.sort_values(by='Net_index')
    


In [ ]:
decile = 100
num_iterations = decile
matrix_7 = np.zeros((7, num_iterations))
for id,index in enumerate(range(1, decile+1)):
    FC_name =  nib.load(f"./R2.2/lesion_network/ROI100/roi_{decile}_{index}_t_fwe_0.001.nii.gz")
    FC_net7 = np.array(Net_7.transform(FC_name)).squeeze()

    N_comparision = N_name.iloc[1:,:]
    R = pd.DataFrame(FC_net7)
    R.index = N_comparision.index
    R.columns = ['R']
    N_comparision = pd.concat([R, R, R, N_comparision], axis= 1)

    # net 7
    for i in range(len(N_comparision)):
        N_comparision.iloc[i,1] = N_comparision.iloc[i,3].split('_')[-2]
        if (N_comparision.iloc[i,1] in ['LH', 'RH']):
            N_comparision.iloc[i,1] = N_comparision.iloc[i,3].split('_')[-1]
        if ('SalVentAttn' in N_comparision.iloc[i,1]):
            N_comparision.iloc[i,1] = N_comparision.iloc[i,1].replace('SalVentAttn', 'VentAttn')
    N_comparision.columns = ['Net_index', 'Name', 'R', 'Label_Name', 'Network_Name','Full_component_name']

    for i in np.unique(N_comparision.loc[:,'Name'].values):
        for j in range(len(N_comparision)):
            if N_comparision.iloc[j,1]== i:
                N_comparision.iloc[j,0]= int(index_7[index_7.loc[:,'None']==i].index.values)
    N_comparision = N_comparision.sort_values(by='Net_index')

    
    N_cur = N_comparision.groupby('Name').mean()
    matrix_7[:, index - 1] = N_cur['R'].values
matrix_7 = pd.DataFrame(matrix_7)
matrix_7.index = N_cur.index
matrix_7.columns = range(1, decile+1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as mp, seaborn

# Define the desired row order
row_order = ['SomMot', 'Vis', 'DorsAttn', 'VentAttn', 'Cont', 'Limbic', 'Default']

# Create a DataFrame with the matrix_data and row_order
heatmap_df = pd.DataFrame(matrix_7, index=row_order)

# Plot the heatmap
seaborn.heatmap(heatmap_df.T, center=0, cmap="RdBu_r")
plt.yticks(range(10, 101, 10), labels=range(10, 101, 10), rotation=0)
plt.title('Seneorimotor to Association')
plt.savefig(f'./R2.2/Output/roi_{decile}_FC2cort_heatmap_net7.png',dpi=500, bbox_inches='tight')
plt.show()